In the name of Allah

## Read data

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [3]:
!cp /content/drive/MyDrive/MLSD/hw1/data/normalized_raw_data.csv /content/

In [2]:
df = pd.read_csv("/content/normalized_raw_data.csv")
df

,id,cat1,cat2,cat3,title,description,url,sub_title,price,size,year,room_number,color,mileage,CPU,RAM
0,1,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک hp,با درود لپ تاپ‌های استوک مارک hp hp probook ۶۵...,https://divar.ir/v/لپ-تاپ-استوک-hp_رایانه-همرا...,۲ ساعت پیش در شیراز، ملاصدرا,14900000,-1,-1,-1,خاکستری,-1,Core i7,8.0
1,2,electronic-devices,computers,laptop-notebook-macbook,لپتاپ HP PRO نسل ۹ مهندسی,لپ تاپ، مهندسی، گرافیکی، پر سرعت شیک و ظریف بر...,https://divar.ir/v/لپتاپ-hp-pro-نسل-۹-مهندسی_ر...,۲ ساعت پیش در شیراز، وصال,18800000,-1,-1,-1,نقره‌ای,-1,Ryzen 5,8.0
2,3,electronic-devices,computers,laptop-notebook-macbook,فروش لپتاپ استوک اروپایی,سلام تکنو رایان واردکننده لپتاپ استوک اروپایی ...,https://divar.ir/v/فروش-لپتاپ-استوک-اروپایی_را...,۳ ساعت پیش در شیراز، زند,3900000,-1,-1,-1,Unknown,-1,Unknown,4.0
3,4,electronic-devices,computers,laptop-notebook-macbook,HP ZBOOK۱۵ g۳ با گرافیک مجزا مناسب طراحی,بدون واسطه به قیمت عمده خرید کنید فروشگاه لپت...,https://divar.ir/v/hp-zbook15-g3-با-گرافیک-مجز...,۳ ساعت پیش در شیراز، ملاصدرا,17900000,-1,-1,-1,مشکی,-1,Core i7,8.0
4,5,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک واردات مستقیم از دبی,با سلام و احترام خدمت شما کاربر گرامی. کمال ت...,https://divar.ir/v/لپ-تاپ-استوک-واردات-مستقیم-...,۳ ساعت پیش در شیراز، پودنک,10000000,-1,-1,-1,سایر,-1,غیره,غیره
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,1424,vehicles,auto,car,پژو پارس پرشیا اتومات فابریک موتور TU۵نقد اقساط,سلام و درود خدمت همه عزیزان نقد و اقساط حداقل ...,https://divar.ir/v/پژو-پارس-پرشیا-اتومات-فابری...,نیم ساعت پیش در تهران، نازی‌آباد,360000000,-1,1389,-1,خاکستری,143000,Unknown,-1
1424,1425,vehicles,auto,car,جیلی GC۶ اکسلنت، مدل ۱۳۹۸,خودرو در حد صفر میباشد اتوماتیک دارای گیربکس ژ...,https://divar.ir/v/جیلی-gc6-اکسلنت-مدل-۱۳۹۸_سو...,نیم ساعت پیش در تهران، پونک,740000000,-1,1398,-1,سفید,60000,Unknown,-1
1425,1426,vehicles,auto,car,ام وی ام ایکس۳۳ATنیوفیس جدید,ام وی ام ایکس۳۳ATنیوفیس بسیار تمیز و بدون خط و...,https://divar.ir/v/ام-وی-ام-ایکس۳۳-at-نیوفیس-ج...,نیم ساعت پیش در تهران، ولنجک,1190000000,-1,1400,-1,خاکستری,56000,Unknown,-1
1426,1427,vehicles,auto,car,پژو ۲۰۶ تیپ ۲، مدل ۱۳۹۸,فوق العاده سالم و بی رنگ یک جفت باند پایونیر چ...,https://divar.ir/v/پژو-206-تیپ-۲-مدل-۱۳۹۸_سوار...,نیم ساعت پیش در تهران، دریاچه شهدای خلیج فارس,468000000,-1,1398,-1,سفید,72000,Unknown,-1


## tf-idf

In [131]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394489 sha256=0b11e73e3495acecbc852c24fad4aff8c85576845c2b0946b475af626fb6c549
  Stored in directory: /root/.cache/pip/wheels/ac/62/f6/88933dadcd64a1614894614aa68cf57c4b8e5256acb650b1f1
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp39-cp39-linux_x86_64.whl size=180376 sha256=1548b89a8a13ab9a8923a1b1d313358b46878a91f369a4ad4724b12874762d4e
  Stored in directory: /root/.cache/pip/wheels/8e/ff/82/9326b96f96f47472e02c453697b225813e4650c0ed4df2cd49
Successfully built nltk libwapiti
  Attempting unin

In [132]:
from sklearn.feature_extraction.text import TfidfVectorizer
from __future__ import unicode_literals
from hazm import *

In [141]:
text = list(df["title"])
text.extend(list(df["description"]))
text.extend(list(df["sub_title"]))

normalizer = Normalizer()
text_normalized = [[normalizer.normalize(y) for y in x.split()] for x in tqdm(text)]
text_sentences = [sent_tokenize(' '.join(x)) for x in tqdm(text_normalized)]
text_tokens = [[word_tokenize(sent) for sent in sents][0] for sents in tqdm(text_sentences)]

vectorizer = TfidfVectorizer(use_idf = True, norm ='l2', analyzer='word')
text_vector = vectorizer.fit_transform([' '.join(doc) for doc in text_tokens])


100%|██████████| 4284/4284 [00:05<00:00, 727.21it/s] 

100%|██████████| 4284/4284 [00:00<00:00, 102368.87it/s]

100%|██████████| 4284/4284 [00:00<00:00, 27929.21it/s]


In [152]:
title_encoding = [doc for doc in text_vector.toarray()[:1428]]
description_encoding = [doc for doc in text_vector.toarray()[1428:1428*2]]
sub_title_encoding = [doc for doc in text_vector.toarray()[1428*2:]]

In [156]:
df.insert(loc=6, column='description_encoding', value=description_encoding)
df.insert(loc=5, column='title_encoding', value=title_encoding)
df.insert(loc=10, column='sub_title_encoding', value=sub_title_encoding)

In [157]:
df

,id,cat1,cat2,cat3,title,title_encoding,description,description_encoding,url,sub_title,sub_title_encoding,price,size,year,room_number,color,mileage,CPU,RAM
0,1,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک hp,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",با درود لپ تاپ‌های استوک مارک hp hp probook ۶۵...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/لپ-تاپ-استوک-hp_رایانه-همرا...,۲ ساعت پیش در شیراز، ملاصدرا,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",14900000,-1,-1,-1,خاکستری,-1,Core i7,8.0
1,2,electronic-devices,computers,laptop-notebook-macbook,لپتاپ HP PRO نسل ۹ مهندسی,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",لپ تاپ، مهندسی، گرافیکی، پر سرعت شیک و ظریف بر...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/لپتاپ-hp-pro-نسل-۹-مهندسی_ر...,۲ ساعت پیش در شیراز، وصال,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",18800000,-1,-1,-1,نقره‌ای,-1,Ryzen 5,8.0
2,3,electronic-devices,computers,laptop-notebook-macbook,فروش لپتاپ استوک اروپایی,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",سلام تکنو رایان واردکننده لپتاپ استوک اروپایی ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/فروش-لپتاپ-استوک-اروپایی_را...,۳ ساعت پیش در شیراز، زند,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3900000,-1,-1,-1,Unknown,-1,Unknown,4.0
3,4,electronic-devices,computers,laptop-notebook-macbook,HP ZBOOK۱۵ g۳ با گرافیک مجزا مناسب طراحی,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",بدون واسطه به قیمت عمده خرید کنید فروشگاه لپت...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/hp-zbook15-g3-با-گرافیک-مجز...,۳ ساعت پیش در شیراز، ملاصدرا,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",17900000,-1,-1,-1,مشکی,-1,Core i7,8.0
4,5,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک واردات مستقیم از دبی,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",با سلام و احترام خدمت شما کاربر گرامی. کمال ت...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/لپ-تاپ-استوک-واردات-مستقیم-...,۳ ساعت پیش در شیراز، پودنک,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10000000,-1,-1,-1,سایر,-1,غیره,غیره
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,1424,vehicles,auto,car,پژو پارس پرشیا اتومات فابریک موتور TU۵نقد اقساط,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",سلام و درود خدمت همه عزیزان نقد و اقساط حداقل ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/پژو-پارس-پرشیا-اتومات-فابری...,نیم ساعت پیش در تهران، نازی‌آباد,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",360000000,-1,1389,-1,خاکستری,143000,Unknown,-1
1424,1425,vehicles,auto,car,جیلی GC۶ اکسلنت، مدل ۱۳۹۸,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",خودرو در حد صفر میباشد اتوماتیک دارای گیربکس ژ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/جیلی-gc6-اکسلنت-مدل-۱۳۹۸_سو...,نیم ساعت پیش در تهران، پونک,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",740000000,-1,1398,-1,سفید,60000,Unknown,-1
1425,1426,vehicles,auto,car,ام وی ام ایکس۳۳ATنیوفیس جدید,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",ام وی ام ایکس۳۳ATنیوفیس بسیار تمیز و بدون خط و...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/ام-وی-ام-ایکس۳۳-at-نیوفیس-ج...,نیم ساعت پیش در تهران، ولنجک,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1190000000,-1,1400,-1,خاکستری,56000,Unknown,-1
1426,1427,vehicles,auto,car,پژو ۲۰۶ تیپ ۲، مدل ۱۳۹۸,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",فوق العاده سالم و بی رنگ یک جفت باند پایونیر چ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/پژو-206-تیپ-۲-مدل-۱۳۹۸_سوار...,نیم ساعت پیش در تهران، دریاچه شهدای خلیج فارس,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",468000000,-1,1398,-1,سفید,72000,Unknown,-1


In [158]:
df.to_json("tfidf_em_normalized_raw_data.json", orient="records")

## BERT embedding

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.8 MB/s eta 0:00:00


In [49]:
from transformers import AutoConfig, AutoTokenizer, AutoModel

model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
bert_model = AutoModel.from_pretrained(model_name_or_path)

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

In [55]:
description_encoding = []
for text in tqdm(df["description"]):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = bert_model(**encoded_input)
    description_encoding.append(output.pooler_output[0].detach().numpy())

100%|██████████| 1428/1428 [07:46<00:00,  3.06it/s]


In [103]:
description_encoding = [t.astype('float64') for t in description_encoding]
df.insert(loc=6, column='description_encoding', value=description_encoding)

In [58]:
title_encoding = []
for text in tqdm(df["title"]):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = bert_model(**encoded_input)
    title_encoding.append(output.pooler_output[0].detach().numpy())

100%|██████████| 1428/1428 [02:46<00:00,  8.57it/s]


In [105]:
title_encoding = [t.astype('float64') for t in title_encoding]
df.insert(loc=5, column='title_encoding', value=title_encoding)

In [59]:
sub_title_encoding = []
for text in tqdm(df["sub_title"]):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = bert_model(**encoded_input)
    sub_title_encoding.append(output.pooler_output[0].detach().numpy())

100%|██████████| 1428/1428 [02:26<00:00,  9.76it/s]


In [108]:
sub_title_encoding = [t.astype('float64') for t in sub_title_encoding]
df.insert(loc=10, column='sub_title_encoding', value=sub_title_encoding)

In [110]:
df

,id,cat1,cat2,cat3,title,title_encoding,description,description_encoding,url,sub_title,sub_title_encoding,price,size,year,room_number,color,mileage,CPU,RAM
0,1,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک hp,"[0.034483104944229126, -0.5618650913238525, 0....",با درود لپ تاپ‌های استوک مارک hp hp probook ۶۵...,"[0.4541394114494324, -0.5242476463317871, -0.6...",https://divar.ir/v/لپ-تاپ-استوک-hp_رایانه-همرا...,۲ ساعت پیش در شیراز، ملاصدرا,"[0.2204636037349701, -0.5848299264907837, 0.18...",14900000,-1,-1,-1,خاکستری,-1,Core i7,8.0
1,2,electronic-devices,computers,laptop-notebook-macbook,لپتاپ HP PRO نسل ۹ مهندسی,"[-0.08948571979999542, -0.28272610902786255, -...",لپ تاپ، مهندسی، گرافیکی، پر سرعت شیک و ظریف بر...,"[0.4279249608516693, -0.09209676831960678, -0....",https://divar.ir/v/لپتاپ-hp-pro-نسل-۹-مهندسی_ر...,۲ ساعت پیش در شیراز، وصال,"[0.18978764116764069, -0.8134634494781494, 0.0...",18800000,-1,-1,-1,نقره‌ای,-1,Ryzen 5,8.0
2,3,electronic-devices,computers,laptop-notebook-macbook,فروش لپتاپ استوک اروپایی,"[0.1311681717634201, -0.4512939155101776, -0.2...",سلام تکنو رایان واردکننده لپتاپ استوک اروپایی ...,"[0.4039764404296875, 0.0694141760468483, -0.52...",https://divar.ir/v/فروش-لپتاپ-استوک-اروپایی_را...,۳ ساعت پیش در شیراز، زند,"[0.19604484736919403, -0.5485450029373169, 0.3...",3900000,-1,-1,-1,Unknown,-1,Unknown,4.0
3,4,electronic-devices,computers,laptop-notebook-macbook,HP ZBOOK۱۵ g۳ با گرافیک مجزا مناسب طراحی,"[-0.010194596834480762, 0.09466830641031265, -...",بدون واسطه به قیمت عمده خرید کنید فروشگاه لپت...,"[0.6834085583686829, -0.5622297525405884, 0.37...",https://divar.ir/v/hp-zbook15-g3-با-گرافیک-مجز...,۳ ساعت پیش در شیراز، ملاصدرا,"[0.1570505052804947, -0.5863929986953735, 0.21...",17900000,-1,-1,-1,مشکی,-1,Core i7,8.0
4,5,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک واردات مستقیم از دبی,"[0.5107795000076294, -0.6868691444396973, -0.6...",با سلام و احترام خدمت شما کاربر گرامی. کمال ت...,"[0.13397206366062164, -0.54062819480896, -0.78...",https://divar.ir/v/لپ-تاپ-استوک-واردات-مستقیم-...,۳ ساعت پیش در شیراز، پودنک,"[0.10386686027050018, -0.5051131248474121, 0.3...",10000000,-1,-1,-1,سایر,-1,غیره,غیره
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,1424,vehicles,auto,car,پژو پارس پرشیا اتومات فابریک موتور TU۵نقد اقساط,"[0.6493532061576843, -0.799845278263092, 0.080...",سلام و درود خدمت همه عزیزان نقد و اقساط حداقل ...,"[0.4858162999153137, -0.19025477766990662, -0....",https://divar.ir/v/پژو-پارس-پرشیا-اتومات-فابری...,نیم ساعت پیش در تهران، نازی‌آباد,"[0.1603245884180069, -0.09049127995967865, -0....",360000000,-1,1389,-1,خاکستری,143000,Unknown,-1
1424,1425,vehicles,auto,car,جیلی GC۶ اکسلنت، مدل ۱۳۹۸,"[0.3157203197479248, -0.9006794095039368, -0.5...",خودرو در حد صفر میباشد اتوماتیک دارای گیربکس ژ...,"[0.6512008309364319, -0.06879565864801407, -0....",https://divar.ir/v/جیلی-gc6-اکسلنت-مدل-۱۳۹۸_سو...,نیم ساعت پیش در تهران، پونک,"[0.2783355414867401, -0.06207304447889328, -0....",740000000,-1,1398,-1,سفید,60000,Unknown,-1
1425,1426,vehicles,auto,car,ام وی ام ایکس۳۳ATنیوفیس جدید,"[0.027393221855163574, -0.5127155780792236, -0...",ام وی ام ایکس۳۳ATنیوفیس بسیار تمیز و بدون خط و...,"[0.6628356575965881, -0.7012090086936951, -0.1...",https://divar.ir/v/ام-وی-ام-ایکس۳۳-at-نیوفیس-ج...,نیم ساعت پیش در تهران، ولنجک,"[0.24986694753170013, -0.21811681985855103, -0...",1190000000,-1,1400,-1,خاکستری,56000,Unknown,-1
1426,1427,vehicles,auto,car,پژو ۲۰۶ تیپ ۲، مدل ۱۳۹۸,"[0.23309168219566345, -0.8837000727653503, 0.0...",فوق العاده سالم و بی رنگ یک جفت باند پایونیر چ...,"[0.5279671549797058, -0.5080827474594116, -0.2...",https://divar.ir/v/پژو-206-تیپ-۲-مدل-۱۳۹۸_سوار...,نیم ساعت پیش در تهران، دریاچه شهدای خلیج فارس,"[0.20848126709461212, -0.4041190445423126, -0....",468000000,-1,1398,-1,سفید,72000,Unknown,-1


In [111]:
df.to_json("bert_em_normalized_raw_data.json", orient="records")

## FastText

In [3]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import fasttext.util

In [5]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz

--2023-03-27 16:47:27--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.225.142.90, 13.225.142.121, 13.225.142.76, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.225.142.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4502524724 (4.2G) [application/octet-stream]
Saving to: ‘cc.fa.300.bin.gz’

cc.fa.300.bin.gz    100%[===================>]   4.19G  49.3MB/s    in 86s     

2023-03-27 16:48:53 (50.1 MB/s) - ‘cc.fa.300.bin.gz’ saved [4502524724/4502524724]



In [8]:
!gunzip /content/cc.fa.300.bin.gz

In [9]:
ft = fasttext.load_model('cc.fa.300.bin')

In [18]:
title_encoding = [ft.get_word_vector(t) for t in df['title']]
description_encoding = [ft.get_word_vector(t) for t in df['description']]
sub_title_encoding = [ft.get_word_vector(t) for t in df['sub_title']]

In [19]:
df.insert(loc=6, column='description_encoding', value=description_encoding)
df.insert(loc=5, column='title_encoding', value=title_encoding)
df.insert(loc=10, column='sub_title_encoding', value=sub_title_encoding)

In [20]:
df

,id,cat1,cat2,cat3,title,title_encoding,description,description_encoding,url,sub_title,sub_title_encoding,price,size,year,room_number,color,mileage,CPU,RAM
0,1,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک hp,"[0.0044191913, 0.0014105368, -0.003266572, 0.0...",با درود لپ تاپ‌های استوک مارک hp hp probook ۶۵...,"[0.0055834227, 0.0084018065, 0.0012574522, 0.0...",https://divar.ir/v/لپ-تاپ-استوک-hp_رایانه-همرا...,۲ ساعت پیش در شیراز، ملاصدرا,"[-0.0017112222, 0.02765921, 0.010748463, 0.012...",14900000,-1,-1,-1,خاکستری,-1,Core i7,8.0
1,2,electronic-devices,computers,laptop-notebook-macbook,لپتاپ HP PRO نسل ۹ مهندسی,"[0.032628242, 0.034881044, 0.011885938, 0.0091...",لپ تاپ، مهندسی، گرافیکی، پر سرعت شیک و ظریف بر...,"[0.008804533, 0.009818383, 0.0022066662, 0.002...",https://divar.ir/v/لپتاپ-hp-pro-نسل-۹-مهندسی_ر...,۲ ساعت پیش در شیراز، وصال,"[0.00692525, 0.026805883, 0.008052798, 0.01132...",18800000,-1,-1,-1,نقره‌ای,-1,Ryzen 5,8.0
2,3,electronic-devices,computers,laptop-notebook-macbook,فروش لپتاپ استوک اروپایی,"[0.0380223, 0.037487496, -0.0030205941, 0.0176...",سلام تکنو رایان واردکننده لپتاپ استوک اروپایی ...,"[0.0066714115, 0.015034727, 0.0036566602, 0.00...",https://divar.ir/v/فروش-لپتاپ-استوک-اروپایی_را...,۳ ساعت پیش در شیراز، زند,"[-0.010540829, 0.020279286, 0.007543174, 0.012...",3900000,-1,-1,-1,Unknown,-1,Unknown,4.0
3,4,electronic-devices,computers,laptop-notebook-macbook,HP ZBOOK۱۵ g۳ با گرافیک مجزا مناسب طراحی,"[0.024284165, 0.036271878, 0.0059037153, 0.007...",بدون واسطه به قیمت عمده خرید کنید فروشگاه لپت...,"[0.0067295157, 0.008296178, 0.0028628968, 0.00...",https://divar.ir/v/hp-zbook15-g3-با-گرافیک-مجز...,۳ ساعت پیش در شیراز، ملاصدرا,"[-0.00706585, 0.030991036, 0.012722894, 0.0145...",17900000,-1,-1,-1,مشکی,-1,Core i7,8.0
4,5,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک واردات مستقیم از دبی,"[0.020904575, -0.0019874193, -0.00658881, 0.01...",با سلام و احترام خدمت شما کاربر گرامی. کمال ت...,"[0.007092952, 0.01360633, 0.001662748, 0.00313...",https://divar.ir/v/لپ-تاپ-استوک-واردات-مستقیم-...,۳ ساعت پیش در شیراز، پودنک,"[-0.006828717, 0.026144609, -1.6363028e-05, 0....",10000000,-1,-1,-1,سایر,-1,غیره,غیره
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,1424,vehicles,auto,car,پژو پارس پرشیا اتومات فابریک موتور TU۵نقد اقساط,"[0.0102948705, -0.002752001, 0.0024168622, 4.3...",سلام و درود خدمت همه عزیزان نقد و اقساط حداقل ...,"[0.008205697, 0.0074793184, -0.0015554602, 0.0...",https://divar.ir/v/پژو-پارس-پرشیا-اتومات-فابری...,نیم ساعت پیش در تهران، نازی‌آباد,"[0.011032805, 0.012933813, 0.0045293, 0.008109...",360000000,-1,1389,-1,خاکستری,143000,Unknown,-1
1424,1425,vehicles,auto,car,جیلی GC۶ اکسلنت، مدل ۱۳۹۸,"[0.008684911, -0.012155267, -0.012916337, 0.00...",خودرو در حد صفر میباشد اتوماتیک دارای گیربکس ژ...,"[0.011510459, 0.01350222, 0.0012081785, 0.0028...",https://divar.ir/v/جیلی-gc6-اکسلنت-مدل-۱۳۹۸_سو...,نیم ساعت پیش در تهران، پونک,"[0.0069503807, 0.01281746, 0.007145154, 0.0072...",740000000,-1,1398,-1,سفید,60000,Unknown,-1
1425,1426,vehicles,auto,car,ام وی ام ایکس۳۳ATنیوفیس جدید,"[0.029646432, 0.0257136, -0.0076587615, 0.0059...",ام وی ام ایکس۳۳ATنیوفیس بسیار تمیز و بدون خط و...,"[0.005119219, 0.0034601572, -0.0015997335, 0.0...",https://divar.ir/v/ام-وی-ام-ایکس۳۳-at-نیوفیس-ج...,نیم ساعت پیش در تهران، ولنجک,"[0.014746906, 0.016231576, 0.00913199, 0.00517...",1190000000,-1,1400,-1,خاکستری,56000,Unknown,-1
1426,1427,vehicles,auto,car,پژو ۲۰۶ تیپ ۲، مدل ۱۳۹۸,"[0.008001841, 0.00028935156, -0.0066452753, 0....",فوق العاده سالم و بی رنگ یک جفت باند پایونیر چ...,"[0.0073550576, 0.003648943, 0.002711118, 0.003...",https://divar.ir/v/پژو-206-تیپ-۲-مدل-۱۳۹۸_سوار...,نیم ساعت پیش در تهران، دریاچه شهدای خلیج فارس,"[0.0014075814, 0.02441383, -0.0040155463, 0.00...",468000000,-1,1398,-1,سفید,72000,Unknown,-1


In [22]:
df.to_json("fasttext_em_normalized_raw_data.json", orient="records")